In [4]:
#scraping data...
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
driver = webdriver.Chrome()
driver.maximize_window()
wait = WebDriverWait(driver, 20)

# Navigate to the Daraz mobile phones section
url = 'https://www.daraz.pk/smartphones/'
driver.get(url)

# List to store all product links
all_product_links = []

# Iterate through the first 5 pages to collect product links
for page_number in range(5):
    time.sleep(2)
    # Wait for the movie grid to be present in the DOM
    ad_grid = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'box--ujueT')))
    
    # Extract links from the current page
    product_links = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')
    time.sleep(1)
    all_product_links.extend([link.find_element(By.TAG_NAME, 'a').get_attribute('href') for link in product_links])

    # Scroll to just above the "Next" button
    driver.execute_script("window.scrollTo(0, 4000);")
    time.sleep(2)
    next_button = driver.find_element(By.CLASS_NAME, 'ant-pagination-next')
   
    next_button.click()
        # Wait for the next page to load

# Print the collected links
print(all_product_links)
all_product_details = []

# Iterate through the collected product links
for product_link in all_product_links:
    driver.get(product_link)
    time.sleep(2)
    # Create a dictionary to store product details
    product_details = {}

    # Add the product link to the dictionary
    product_details['Link'] = product_link
    # Extract product details
    try:
        product_details['Title'] = driver.find_element(By.CLASS_NAME, 'pdp-mod-product-badge-title').text
    except NoSuchElementException:
        product_details['Title'] = None
    
    try:
        product_details['Price'] = driver.find_element(By.CLASS_NAME, 'pdp-product-price').text
    except NoSuchElementException:
        product_details['Price'] = None
    
    try:
        time.sleep(1)
        product_details['Brand'] = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[5]/div/a[1]').text
    except NoSuchElementException:
        product_details['Brand'] = None
    try:
        time.sleep(1)
        product_details['Specifications'] = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[12]/div/div[2]/div/div/div[1]/span').text
    except NoSuchElementException as e:
        product_details['Specifications'] = None
    try:
        driver.execute_script("window.scrollTo(0, 700);")
        time.sleep(2)
        product_details['Rating'] =(float(driver.find_element(By.XPATH, '/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[2]/div[1]/div[1]/div[1]/span[1]').text))
    except:
        product_details['Rating'] = None
    try:
        driver.execute_script("window.scrollTo(800, 1000);")
        time.sleep(2)
        reviews_container = driver.find_element(By.CLASS_NAME, 'review-content')
        reviews = reviews_container.find_elements(By.CLASS_NAME, 'review-item')
        
        # Extract individual reviews
        reviews_data = []
        for review in reviews:
            time.sleep(2)
            review_data = {
                'Name': review.find_element(By.CLASS_NAME, 'user').text,
                'Review': review.find_element(By.CLASS_NAME, 'review-content-sl').text
            }
            reviews_data.append(review_data)
        
        product_details['Reviews'] = reviews_data
    except NoSuchElementException:
        product_details['Reviews'] = None
    
    # Append the product details to the list
    all_product_details.append(product_details)

# Create a DataFrame with the collected product details
df_product_details = pd.DataFrame(all_product_details)

# Display or save the DataFrame as needed
print(df_product_details)

# Close the browser
driver.quit()

['https://www.daraz.pk/products/samsung-a14-4gb64gb-5000-mah-battery-66-display-android-13-i436839264-s2101404094.html?search=1', 'https://www.daraz.pk/products/redmi-note-12-8-gb-ram-128gb-rom-pta-approved-mobile-phone-with-1-year-brand-warranty-i429185340-s2040915791.html?search=1', 'https://www.daraz.pk/products/infinix-note-30-pro-8-256-gb-pta-approved-with-official-1-year-warranty-i436821010-s2101334479.html?search=1', 'https://www.daraz.pk/products/redmi-a2-3gb64gb-mediatek-helio-g36-android-12-5000-mah-battery-i430451931-s2051820850.html?search=1', 'https://www.daraz.pk/products/realme-narzo-50a-prime-4gb-128gb-pta-approved-with-official-1-year-warranty-i436801398-s2101274809.html?search=1', 'https://www.daraz.pk/products/redmi-note-12-8gb-ram-128gb-rom-pta-approved-official-warranty-super-fast-charging-i421521093-s1991845039.html?search=1', 'https://www.daraz.pk/products/infinix-note-30-8gb128gb-android-13-678-display-5000-mah-battery-i436820987-s2101357944.html?search=1', 'htt

In [5]:
df_product_details.to_csv('daraz_store.csv', index=False)

In [6]:
import re
csv_file_path = 'daraz_store.csv'
df = pd.read_csv(csv_file_path)

# Clean the 'Price' column by extracting the numeric values after "Rs."
df['Price'] = df['Price'].apply(lambda x: re.findall(r'Rs\. (\d[\d,]*)', str(x)))

# Take the first value if multiple prices are present (e.g., Rs. 50,499\nRs. 51,999-3%)
df['Price'] = df['Price'].apply(lambda x: float(x[0].replace(',', '')) if x else None)

# Convert the 'Price' column to numeric
df['Price'] = pd.to_numeric(df['Price'])

# Save the updated DataFrame to a new CSV file
output_csv_path = 'daraz_store_cleaned.csv'
df.to_csv(output_csv_path, index=False)

df = pd.read_csv('daraz_store_cleaned.csv')
df

,Link,Title,Price,Brand,Specifications,Rating,Reviews
0,https://www.daraz.pk/products/samsung-a14-4gb6...,Samsung A14 4GB/64GB - 5000 mAh Battery - 6.6'...,34999.0,No Brand,64GB,4.4,"[{'Name': 'Muhammad B.', 'Review': ""Unbelievab..."
1,https://www.daraz.pk/products/redmi-note-12-8-...,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,49999.0,Xiaomi,128GB,4.7,"[{'Name': 'Gohar I.', 'Review': 'this is the i..."
2,https://www.daraz.pk/products/infinix-note-30-...,Infinix Note 30 Pro 8-256 GB PTA Approved With...,64499.0,No Brand,256GB,4.7,"[{'Name': 'Snake E.', 'Review': 'I got what wa..."
3,https://www.daraz.pk/products/redmi-a2-3gb64gb...,Redmi A2+ - 3GB/64GB - Mediatek Helio G36 - An...,19399.0,Xiaomi,64GB,4.7,"[{'Name': '3***8', 'Review': 'Alhamdulillah sa..."
4,https://www.daraz.pk/products/realme-narzo-50a...,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,32999.0,No Brand,NaN,4.2,"[{'Name': 'Habib R.', 'Review': '#11.11 PTA ap..."
...,...,...,...,...,...,...,...
195,https://www.daraz.pk/products/7-128gb-8gb-5000...,Sparx Neo 7 Ultra 128GB ROM 8GB RAM Battery: 5...,34999.0,No Brand,128GB,NaN,NaN
196,https://www.daraz.pk/products/samsung-galaxy-a...,"Samsung Galaxy A33 5G - 6.4"" Inch Display - 6G...",99999.0,No Brand,128GB,4.1,"[{'Name': 'Tahir M.', 'Review': 'Acha hai orig..."
197,https://www.daraz.pk/products/30-8gb-128gb-i43...,Infinix Hot 30 8GB Ram 128GB Memory,35989.0,No Brand,128GB,4.6,"[{'Name': '3***3', 'Review': 'very good👍 phone..."
198,https://www.daraz.pk/products/galaxy-a04s-4gb-...,Galaxy A04s 4GB RAM-128GB ROM PTA APPROVED OFF...,38999.0,Samsung,128GB,3.0,"[{'Name': 'SHAHID A.', 'Review': 'gr8 service...."


In [11]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd

# Function to create a connection to the database
def create_db_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='12345678',  
            database='daraz'
        )
        return connection
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Error: Access denied.")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Error: Database does not exist.")
        else:
            print(f"Error: {err}")
        return None

# Function to create tables in the database
def create_tables(connection):
    cursor = connection.cursor()

    # Create Products table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Products (
            ID INT AUTO_INCREMENT PRIMARY KEY,
            Title VARCHAR(255),
            Price VARCHAR(50),
            Brand VARCHAR(100),
            Specifications VARCHAR(255),
            Rating FLOAT
        )
    """)

    # Create Reviews table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Reviews (
            ID INT AUTO_INCREMENT PRIMARY KEY,
            Product_ID INT,
            Name VARCHAR(100),
            Review_Text TEXT,
            FOREIGN KEY (Product_ID) REFERENCES Products(ID)
        )
    """)

    connection.commit()
    cursor.close()

# Function to insert data into the database
def insert_data_into_database(connection, product_details):
    cursor = connection.cursor()

    for _, product in product_details.iterrows():
        # Insert product details into the Products table without the Link column
        cursor.execute("""
            INSERT INTO Products (Title, Price, Brand, Specifications, Rating) VALUES (%s, %s, %s, %s, %s)
        """, (product['Title'], product['Price'], product['Brand'], product['Specifications'], product['Rating']))

        # Get the last inserted ID (Product_ID) for the Reviews table
        product_id = cursor.lastrowid

        # Insert reviews into the Reviews table
        if 'Reviews' in product and product['Reviews']:
            for review in product['Reviews']:
                cursor.execute("""
                    INSERT INTO Reviews (Product_ID, Name, Review_Text) VALUES (%s, %s, %s)
                """, (product_id, review['Name'], review['Review']))

    connection.commit()
    cursor.close()

# Connect to the database
connection = create_db_connection()

if connection:
    # Create tables if they don't exist
    create_tables(connection)

    # Insert data into the database
    insert_data_into_database(connection, df_product_details)

    # Close the database connection
    connection.close()
